<h1> Autoencoders</h1>

#### Installation
    
requirement.txt can be found in the repository. In order install all necessary packages you should run:

pip install -r requirements.txt

Adding Calina library to pythonpath is also necessary. If you are using using anaconda you can just type:

conda develop Calina_path

#### Importing required modules

In [38]:
import os
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from datetime import datetime
datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
import plotly.express as px
import pandas as pd
import neptune.new as neptune
import zipfile

#### networks.py contains our custom architectures for autoencoders. They are built upon the pl.LightningModule

In [2]:
from networks import VeloDecoder, VeloEncoder, VeloAutoencoderLt
from calina_dataset.calibration_dataset import Tell1Dataset

#### Below are all the necessary parameters for the training process, creating plots and saving them in Neptune.

In [3]:
#trainig parameters
PARAMS = {'max_epochs': 50,
          'learning_rate': 0.02,
          'batch_size': 64,
          'gpus': 1,
          'experiment_name': 'small-net more-epochs standarized SGD no-dropout bigger-batches relu shuffle',
          'tags': ['small-net', 'more-epochs', 'standarized', 'SGD', 'no-dropout', 'bigger-batches', 'relu', 'shuffle'],
          'source_files': ['analyze.ipynb', 'networks.py'],
          'experiment_id': 'VEL-371'
}

#### Class for extracting the data.

In [4]:
class MyDS(Tell1Dataset):
    filename_format = '%Y-%m-%d'
    filename_regex_format = r'\d{4}-\d{2}-\d{2}.csv'

#### Loading the data from all sources and channels. We also standardize the input data.

In [6]:
datapath = os.path.join("../../data", "calibrations")
data_list = MyDS.get_filepaths_from_dir(datapath)
mds = MyDS(data_list, read=True)

dfh = mds.dfh.df.iloc[:, 9:]
print(dfh)
dfh_r = mds.dfh['R'].df.iloc[:, 9:]
dfh_phi = mds.dfh['phi'].df.iloc[:, 9:]
dfp = mds.dfp.df.iloc[:, 9:]
dfp_r = mds.dfp['R'].df.iloc[:, 9:]
dfp_phi = mds.dfp['phi'].df.iloc[:, 9:]

dfh_metadata = mds.dfh.df.iloc[:, :9]
dfh_r_metadata = mds.dfh['R'].df.iloc[:, :9]
dfh_phi_metadata = mds.dfh['phi'].df.iloc[:, :9]
dfp_metadata = mds.dfp.df.iloc[:, :9]
dfp_r_metadata = mds.dfp['R'].df.iloc[:, :9]
dfp_phi_metadata = mds.dfp['phi'].df.iloc[:, :9]

scaler = preprocessing.StandardScaler()
dfh_scaled = scaler.fit_transform(dfh)
dfh_r_scaled = scaler.fit_transform(dfh_r)
dfh_phi_scaled = scaler.fit_transform(dfh_phi)
dfp_scaled = scaler.fit_transform(dfp)
dfp_r_scaled = scaler.fit_transform(dfp_r)
dfp_phi_scaled = scaler.fit_transform(dfp_phi)

dfh = pd.DataFrame(dfh_scaled, index=dfh.index, columns=dfh.columns)
dfh_r = pd.DataFrame(dfh_r_scaled, index=dfh_r.index, columns=dfh_r.columns)
dfh_phi = pd.DataFrame(dfh_phi_scaled, index=dfh_phi.index, columns=dfh_phi.columns)
dfp = pd.DataFrame(dfp_scaled, index=dfp.index, columns=dfp.columns)
dfp_r = pd.DataFrame(dfp_r_scaled, index=dfp_r.index, columns=dfp_r.columns)
dfp_phi = pd.DataFrame(dfp_phi_scaled, index=dfp_phi.index, columns=dfp_phi.columns)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:04<00:00,  7.42it/s]


      channel0  channel1  channel2  channel3  channel4  channel5  channel6  \
2         17.0      11.0      11.0      11.0      11.0      11.0      11.0   
5         17.0      11.0      11.0      11.0      11.0      11.0      11.0   
8         18.0      11.0      11.0      11.0      11.0      11.0      11.0   
11        16.0      11.0      11.0      10.0      10.0      11.0      10.0   
14        10.0      11.0      10.0      11.0      11.0      10.0      10.0   
...        ...       ...       ...       ...       ...       ...       ...   
7547      10.0      11.0      11.0      11.0      11.0      11.0      10.0   
7550       9.0      11.0      11.0      11.0      11.0      11.0      10.0   
7553       9.0      11.0      11.0      11.0      11.0      11.0      10.0   
7556      10.0      11.0      11.0      11.0      10.0      11.0      11.0   
7559       9.0      11.0      10.0      11.0      11.0      11.0      11.0   

      channel7  channel8  channel9  ...  channel2038  channel20

#### Creating loaders for the training.

In [7]:
def make_loader(dataset):
    train, test = train_test_split(dataset, test_size=0.2)
    train_target = torch.tensor(train.values, dtype=torch.float)
    train_data = torch.tensor(train.values, dtype=torch.float)
    test_target = torch.tensor(test.values, dtype=torch.float)
    test_data = torch.tensor(test.values, dtype=torch.float)
    train_tensor = TensorDataset(train_data, train_target)
    test_tensor = TensorDataset(test_data, test_target)
    train_loader = DataLoader(dataset=train_tensor, shuffle=True)
    test_loader = DataLoader(dataset=test_tensor, shuffle=True)
    return train_loader, test_loader

#### Creating trainers for the network, with some set training parameters.

In [8]:
def make_model_trainer(s, neptune_logger, lr):
    #s = 2048
    dec = VeloDecoder(s)
    enc = VeloEncoder(s)
    model = VeloAutoencoderLt(enc, dec, lr)
    lr_monitor = LearningRateMonitor(logging_interval='step')
    tr = pl.Trainer(logger=neptune_logger, callbacks=[lr_monitor],  max_epochs=PARAMS['max_epochs'],
                    gpus=PARAMS['gpus'])
    return model, tr

#### Function creating the slider plot to see the evolution of autoencoder results

In [9]:
def slider_plot(dataset, datasetName, metadata, model):
    reducedData = model.enc.forward(torch.tensor(dataset.values, dtype=torch.float))
    reducedData = reducedData.detach().numpy()
     
    indexesList = metadata.index.values.tolist()
    xyDF = pd.DataFrame(reducedData, index=indexesList, columns=['x', 'y'])
    resultDF = pd.concat([metadata, xyDF], axis=1)
    resultDF["datetime"] = resultDF["datetime"].astype(str)

    fig = px.scatter(resultDF, x="x", y="y", animation_frame="datetime", animation_group="sensor", color="sensor")
    fig["layout"].pop("updatemenus") # optional, drop animation buttons
    fig.update_xaxes(range=[1.15*resultDF['x'].min(), 1.15*resultDF['y'].max()])
    fig.update_yaxes(range=[1.15*resultDF['x'].min(), 1.15*resultDF['y'].max()])
    fig.show()
    return fig

#### Function creating the plot being the result of the training. All the channels can be later turned on/off in a custom way

In [10]:
def clustering_plot(dataset, datasetName, metadata, model):
    
    reducedData = model.enc.forward(torch.tensor(dataset.values, dtype=torch.float))
    reducedData = reducedData.detach().numpy()
     
    indexesList = metadata.index.values.tolist()
    xyDF = pd.DataFrame(reducedData, index=indexesList, columns=['x', 'y'])
    resultDF = pd.concat([metadata, xyDF], axis=1)
    resultDF["datetime"] = resultDF["datetime"].astype(str)
    resultDF["sensor"] = resultDF["sensor"].astype(str)
    print(resultDF)
    
    fig = px.scatter(resultDF, x="x", y="y", color='sensor', opacity=0.5)
    fig.show(renderer="notebook")
    return fig

#### Function connects to Neptune, starts a new experiment there and starts running the training process for the network. It then creates all the plots and saves them in the Neptune as well.

In [11]:
def run_experiment(dataset, datasetName, par, metadata):
    model_path = os.path.join('models', par['experiment_name'], datasetName)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
        
    train_loader, test_loader = make_loader(dataset)
    s = dataset.shape[1]
    neptune_logger = NeptuneLogger(
        api_key=os.getenv('NEPTUNE_API_TOKEN'),
        project_name="pawel-drabczyk/velodimred",
        experiment_name=par['experiment_name'],
        params=par,
        tags=par['tags'] + [datasetName] + ['interactive'],
        upload_source_files=par['source_files']
    )
    model, tr = make_model_trainer(s, neptune_logger, par['learning_rate'])
    tr.fit(model, train_loader, test_loader)

    torch.save(model, os.path.join('models', PARAMS['experiment_name'], datasetName, "trained_model.ckpt"))
    neptune_logger.experiment.log_artifact(os.path.join(model_path, "trained_model.ckpt"))
    
    fig = slider_plot(dataset, datasetName, metadata, model)
    fig.write_html(os.path.join(model_path, 'slider_plot.html'))
    fig.write_image(os.path.join(model_path, 'slider_plot.png'))   
    neptune_logger.experiment.log_image('slider_plot',os.path.join(model_path, 'slider_plot.png'))    
    fig = clustering_plot(dataset, datasetName, metadata, model)
    fig.write_html(os.path.join(model_path, 'clustering_plot.html'))
    fig.write_image(os.path.join(model_path, 'clustering_plot.png'))    
    neptune_logger.experiment.log_image('clustering_plot', os.path.join(model_path, 'clustering_plot.png'))        
    
    neptune_logger.experiment.log_artifact(os.path.join(model_path, "slider_plot.html"))
    neptune_logger.experiment.log_artifact(os.path.join(model_path, "clustering_plot.html"))

#### The function was used to reopen experiments, and add the cluster plots and slider plots, which was added at the end of the project.

In [43]:
def reopen_experiment(dataset, datasetName, par, metadata):
    exp_id = par['experiment_id']
    exp_name = par['experiment_name']
    model_path = os.path.join('models', exp_name, datasetName)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    
    run = neptune.init('pawel-drabczyk/velodimred', run=PARAMS['experiment_id'])
    run['artifacts/trained_model.ckpt'].download()
    with zipfile.ZipFile("files.zip","r") as zip_ref:
        zip_ref.extractall()
    from networks import VeloDecoder, VeloEncoder, VeloAutoencoderLt
        
    model = torch.load(os.path.join(model_path,'trained_model.ckpt'))
    
    fig = slider_plot(dataset, datasetName, metadata, model)
    fig.write_html(os.path.join(model_path, 'slider_plot.html'))
    fig.write_image(os.path.join(model_path, 'slider_plot.png'))   
    my_exp.log_image('slider_plot',os.path.join(model_path, 'slider_plot.png'))    
    fig = clustering_plot(dataset, datasetName, metadata, model)
    fig.write_html(os.path.join(model_path, 'clustering_plot.html'))
    fig.write_image(os.path.join(model_path, 'clustering_plot.png'))    
    my_exp.log_image('clustering_plot', os.path.join(model_path, 'clustering_plot.png'))        
    
    my_exp.log_artifact(os.path.join(model_path, "slider_plot.html"))
    my_exp.log_artifact(os.path.join(model_path, "clustering_plot.html"))
    my_exp.append_tag('interactive')

#### The cell below runs te training process for all the datasets, for the current experiment, for the current network configuration.

In [44]:
datasetNames = ['dfh', 'dfhr', 'dfhphi', 'dfp', 'dfpr', 'dfpphi']

#run_experiment(dfh, 'dfh', PARAMS, dfh_metadata)
#run_experiment(dfh_r, 'dfhr', PARAMS, dfh_r_metadata)
#run_experiment(dfh_phi, 'dfhphi', PARAMS, dfh_phi_metadata)
#run_experiment(dfp, 'dfp', PARAMS, dfp_metadata)
#run_experiment(dfp_r, 'dfpr', PARAMS, dfp_r_metadata)
#run_experiment(dfp_phi, 'dfpphi', PARAMS, dfp_phi_metadata)

#### The cell below opens existing experiment, adding the slider plots and cluster plots.

In [45]:
!source .envrc

In [47]:
#reopen_experiment(dfh, 'dfh', PARAMS, dfh_metadata)
reopen_experiment(dfh_r, 'dfhr', PARAMS, dfh_r_metadata)
#reopen_experiment(dfh_phi, 'dfhphi', PARAMS, dfh_phi_metadata)
#reopen_experiment(dfp, 'dfp', PARAMS, dfp_metadata)
#reopen_experiment(dfp_r, 'dfpr', PARAMS, dfp_r_metadata)
#reopen_experiment(dfp_phi, 'dfpphi', PARAMS, dfp_phi_metadata)

psutil is not installed. Hardware metrics will not be collected.


https://app.neptune.ai/pawel-drabczyk/velodimred/e/VEL-371
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


ImportError: cannot import name 'HOROVOD_AVAILABLE' from 'pytorch_lightning.utilities' (/home/mwm/anaconda3/envs/dimred/lib/python3.8/site-packages/pytorch_lightning/utilities/__init__.py)

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stdout. Invalid point: 2021-08-19T13:13:00.357Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stdout. Invalid point: 2021-08-19T13:13:00.357Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stdout. Invalid point: 2021-08-19T13:13:00.357Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stdout. Invalid point: 2021-08-19T13:13:00.357Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:03.984Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point:

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:15.764Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:15.768Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:15.749Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:15.753Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:15.757Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point:

...

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:34.953Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.833Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:37.021Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:34.955Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.835Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point:

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:34.978Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.870Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:37.080Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:34.980Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.873Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point:

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.906Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:37.122Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.001Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.911Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 202

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.025Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.949Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:37.164Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.026Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.952Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point:

Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.050Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.987Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:37.205Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.051Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point: 2021-08-19T13:13:35.989Z
Error occurred during asynchronous operation processing: Timestamp must be non-decreasing for series attribute: monitoring/stderr. Invalid point:

In [29]:
!ls

analyze.ipynb  Article.html  networks.py  PCA.py       requirements.in
analyze.py     files.zip     PCA.html	  __pycache__
article        models	     PCA.ipynb	  readme.md
